# Day 5

## Part 1

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:
```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:
```
50 98 2
52 50 48
```
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:
```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```
With this map, you can look up the soil number required for each initial seed number:

- Seed number 79 corresponds to soil number 81.
- Seed number 14 corresponds to soil number 14.
- Seed number 55 corresponds to soil number 57.
- Seed number 13 corresponds to soil number 13.
The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

- Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
- Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
- Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
- Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [1]:
class AgricultureMappings:
    def __init__(self, inputs: list[str]):
        # pull out the seed numbers
        self.seeds = self.gather_seeds(inputs[0])

        # configure all the mappings from
        #   seeds->soil->fertilizer->water->light->temperature->humidity->location
        self.mapping: dict = {}
        for mapping in inputs[1:]:
            mapping = mapping.split("\n")
            name = mapping[0].split(":")[0].split()[0]
            mapping = mapping[1:]
            self.mapping[name] = {}
            for line in mapping:
                vals = line.split(" ")
                dest = int(vals[0])
                source = int(vals[1])
                count = int(vals[2])
                self.mapping[name][(source, source + count - 1)] = (dest, dest + count - 1)

    def gather_seeds(self, seed_str: str) -> None:
        # "seeds: 79 14 55 13"
        seed_ids = seed_str.split(": ")[1].split(" ")
        return [int(seed) for seed in seed_ids]

    def get_updated_value_from_mapping(self, value: int, mapping_name: str) -> int:
        new_value = value
        for (start, end), (dest_start, dest_end) in self.mapping[mapping_name].items():
            if start <= value <= end:
                # source/dest ranges were adjusted by the same (count) amount,
                # so we can use that to adjust the value here
                offset = dest_start - start
                new_value = value + offset
                print(
                    f"{value=} @ {mapping_name}: in range ({start}, {end}) adjusted by {offset} for range ({dest_start}, {dest_end}) -> {new_value=}"
                )
                break
        return new_value

    def get_location_for_seed(self, seed: int) -> int:
        soil = self.get_updated_value_from_mapping(seed, mapping_name="seed-to-soil")
        fertilizer = self.get_updated_value_from_mapping(soil, mapping_name="soil-to-fertilizer")
        water = self.get_updated_value_from_mapping(fertilizer, mapping_name="fertilizer-to-water")
        light = self.get_updated_value_from_mapping(water, mapping_name="water-to-light")
        temperature = self.get_updated_value_from_mapping(
            light, mapping_name="light-to-temperature"
        )
        humidity = self.get_updated_value_from_mapping(
            temperature, mapping_name="temperature-to-humidity"
        )
        location = self.get_updated_value_from_mapping(
            humidity, mapping_name="humidity-to-location"
        )
        print(
            f"{seed=} -> {soil=} -> {fertilizer=} -> {water=} -> {light=} -> {temperature=} -> {humidity=} -> {location=}"
        )
        return location

    def get_seed_locations(self) -> dict[int, int]:
        locations = {}
        for seed in self.seeds:
            locations[seed] = self.get_location_for_seed(seed)
        return locations


test_inputs = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""".split(
    "\n\n"
)

test_agmap = AgricultureMappings(test_inputs)
print(test_agmap.seeds)
print(test_agmap.mapping)
print("=" * 10)
test_locations = test_agmap.get_seed_locations()
assert test_locations == {
    79: 82,
    14: 43,
    55: 86,
    13: 35,
}

[79, 14, 55, 13]
{'seed-to-soil': {(98, 99): (50, 51), (50, 97): (52, 99)}, 'soil-to-fertilizer': {(15, 51): (0, 36), (52, 53): (37, 38), (0, 14): (39, 53)}, 'fertilizer-to-water': {(53, 60): (49, 56), (11, 52): (0, 41), (0, 6): (42, 48), (7, 10): (57, 60)}, 'water-to-light': {(18, 24): (88, 94), (25, 94): (18, 87)}, 'light-to-temperature': {(77, 99): (45, 67), (45, 63): (81, 99), (64, 76): (68, 80)}, 'temperature-to-humidity': {(69, 69): (0, 0), (0, 68): (1, 69)}, 'humidity-to-location': {(56, 92): (60, 96), (93, 96): (56, 59)}}
value=79 @ seed-to-soil: in range (50, 97) adjusted by 2 for range (52, 99) -> new_value=81
value=81 @ water-to-light: in range (25, 94) adjusted by -7 for range (18, 87) -> new_value=74
value=74 @ light-to-temperature: in range (64, 76) adjusted by 4 for range (68, 80) -> new_value=78
value=78 @ humidity-to-location: in range (56, 92) adjusted by 4 for range (60, 96) -> new_value=82
seed=79 -> soil=81 -> fertilizer=81 -> water=81 -> light=74 -> temperature=78

In [2]:
inputs = open("../inputs/5.txt").read().split("\n\n")
agmap = AgricultureMappings(inputs)
print(f"seeds={agmap.seeds}")
print(f"mapping={agmap.mapping}")
locations = agmap.get_seed_locations()
locations

seeds=[4088478806, 114805397, 289354458, 164506173, 1415635989, 166087295, 1652880954, 340945548, 3561206012, 483360452, 35205517, 252097746, 1117825174, 279314434, 3227452369, 145640027, 2160384960, 149488635, 2637152665, 236791935]
mapping={'seed-to-soil': {(2926455387, 3381518554): (3333452986, 3788516153), (1807198589, 1918358601): (3222292973, 3333452985), (1120843626, 1342615893): (4073195028, 4294967295), (2255546991, 2262607222): (3215232741, 3222292972), (2727928910, 2760573309): (1658311530, 1690955929), (1918358602, 2255546990): (2680271553, 3017459941), (3973557555, 4002147450): (1690955930, 1719545825), (4046183137, 4294967295): (2081345351, 2330129509), (3613106716, 3919213072): (2374165196, 2680271552), (2504868379, 2553871713): (1553535599, 1602538933), (3919213073, 3973557554): (4018850546, 4073195027), (2287201502, 2317832933): (2050713919, 2081345350), (1775307867, 1807198588): (3183342019, 3215232740), (2553871714, 2597170660): (3975551599, 4018850545), (1342615894,

{4088478806: 3014755458,
 114805397: 1690141612,
 289354458: 925969269,
 164506173: 2452872251,
 1415635989: 2253918459,
 166087295: 2454453373,
 1652880954: 4027561917,
 340945548: 977560359,
 3561206012: 3399734119,
 483360452: 2023666033,
 35205517: 3258643866,
 252097746: 4008609769,
 1117825174: 382895070,
 279314434: 915929245,
 3227452369: 2638545227,
 145640027: 3730151367,
 2160384960: 4171407620,
 149488635: 3733999975,
 2637152665: 641984578,
 236791935: 3993303958}

In [3]:
min(locations.values())

382895070

## Part 2

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [4]:
inputs[0]

'seeds: 4088478806 114805397 289354458 164506173 1415635989 166087295 1652880954 340945548 3561206012 483360452 35205517 252097746 1117825174 279314434 3227452369 145640027 2160384960 149488635 2637152665 236791935'

In [21]:
from rich import print_json, print as rprint
import pprint


class SeedyAgricultureMappings:
    def __init__(self, inputs: list[str]):
        # pull out the seed numbers
        self.seed_ranges = self.gather_seed_ranges(inputs[0])

        # configure all the mappings from
        #   seeds->soil->fertilizer->water->light->temperature->humidity->location
        self.mapping: dict = {}
        for input_str in inputs[1:]:
            input_data = input_str.split("\n")
            name = input_data[0].split(":")[0].split()[0]
            mapping_data = input_data[1:]

            self.mapping[name] = {}
            for data in mapping_data:
                vals = data.split(" ")
                dest = int(vals[0])
                source = int(vals[1])
                count = int(vals[2])
                self.mapping[name][(source, source + count - 1)] = (dest, dest + count - 1)

    def gather_seed_ranges(self, seed_str: str) -> None:
        # "seeds: 4088478806 114805397 289354458 164506173 ..."
        seed_data = seed_str.split(": ")[1].split(" ")
        seed_ranges = []
        for i in range(0, len(seed_data), 2):
            start_seed = int(seed_data[i])
            count = int(seed_data[i + 1])
            seed_ranges.append((start_seed, start_seed + count - 1))
        return seed_ranges

    # def get_updated_value_from_mapping(self, value: int, mapping_name: str) -> int:
    #     new_value = value
    #     for (start, end), (dest_start, dest_end) in self.mapping[mapping_name].items():
    #         if start <= value <= end:
    #             # source/dest ranges were adjusted by the same (count) amount,
    #             # so we can use that to adjust the value here
    #             offset = dest_start - start
    #             new_value = value + offset
    #             print(
    #                 f"{value=} @ {mapping_name}: in range ({start}, {end}) adjusted by {offset} for range ({dest_start}, {dest_end}) -> {new_value=}"
    #             )
    #             break
    #     return new_value

    def get_updated_ranges_from_mapping(
        self, ranges: list(tuple[int, int]), mapping_name: str
    ) -> list[tuple[int, int]]:
        new_ranges = []
        mapping = self.mapping[mapping_name]

        for input_range in ranges:
            input_start, input_end = input_range

            new_input_ranges = []
            for (source_start, source_end), (dest_start, dest_end) in mapping.items():
                intersect_start = max(input_start, source_start)
                intersect_end = min(input_end, source_end)

                if intersect_start <= intersect_end:
                    offset = dest_start - source_start
                    output_start = intersect_start + offset
                    output_end = intersect_end + offset
                    new_input_ranges.append((output_start, output_end))
                elif input_start <= source_end or input_end >= source_start:
                    # unmapped values that partially overlap the mapping ranges
                    if input_start < source_start:
                        new_input_ranges.append((input_start, source_start - 1))
                    if input_end > source_end:
                        new_input_ranges.append((source_end + 1, input_end))

            if new_input_ranges:
                # merge any ranges that might be overlapping, starting by sorting them in order
                # based on their start value
                sorted_ranges = sorted(new_input_ranges, key=lambda x: x[0])

                first_range = sorted_ranges[0]
                last_range = sorted_ranges[-1]

                merged = [first_range]
                # check if the current range overlaps the last merged range
                for current in sorted_ranges[1:]:
                    if current[0] <= last_range[1] + 1:
                        merged[-1] = (last_range[0], max(last_range[1], current[1]))
                    else:
                        merged.append(current)
            else:
                # return the original input range if nothing was mapped
                new_input_ranges.append((input_start, input_end))

            new_ranges += new_input_ranges

        return new_ranges

    def get_seed_location_ranges(self) -> list:
        for seed_range in self.seed_ranges:
            print(f"* {seed_range=}")
            soil_ranges = self.get_updated_ranges_from_mapping(
                [seed_range], mapping_name="seed-to-soil"
            )
            print(f"** {soil_ranges=}")
            fertilizer_ranges = self.get_updated_ranges_from_mapping(
                soil_ranges, mapping_name="soil-to-fertilizer"
            )
            print(f"*** {fertilizer_ranges=}")
            water_ranges = self.get_updated_ranges_from_mapping(
                fertilizer_ranges, mapping_name="fertilizer-to-water"
            )
            print(f"**** {water_ranges=}")
            light_ranges = self.get_updated_ranges_from_mapping(
                water_ranges, mapping_name="water-to-light"
            )
            print(f"***** {light_ranges=}")
            temperature_ranges = self.get_updated_ranges_from_mapping(
                light_ranges, mapping_name="light-to-temperature"
            )
            print(f"****** {temperature_ranges=}")
            humidity_ranges = self.get_updated_ranges_from_mapping(
                temperature_ranges, mapping_name="temperature-to-humidity"
            )
            print(f"******* {humidity_ranges=}")
            location_ranges = self.get_updated_ranges_from_mapping(
                humidity_ranges, mapping_name="humidity-to-location"
            )
            print(f"******** {location_ranges=}")
        return location_ranges


test_agmap2 = SeedyAgricultureMappings(test_inputs)
print(f"seed_ranges={test_agmap2.seed_ranges}")
rprint(test_agmap2.mapping)

test_location_ranges = test_agmap2.get_seed_location_ranges()
test_location_ranges

seed_ranges=[(79, 92), (55, 67)]


{
    'seed-to-soil': {(98, 99): (50, 51), (50, 97): (52, 99)},
    'soil-to-fertilizer': {(15, 51): (0, 36), (52, 53): (37, 38), (0, 14): (39, 53)},
    'fertilizer-to-water': {(53, 60): (49, 56), (11, 52): (0, 41), (0, 6): (42, 48), (7, 10): (57, 60)},
    'water-to-light': {(18, 24): (88, 94), (25, 94): (18, 87)},
    'light-to-temperature': {(77, 99): (45, 67), (45, 63): (81, 99), (64, 76): (68, 80)},
    'temperature-to-humidity': {(69, 69): (0, 0), (0, 68): (1, 69)},
    'humidity-to-location': {(56, 92): (60, 96), (93, 96): (56, 59)}
}

* seed_range=(79, 92)
** soil_ranges=[(79, 97), (81, 94)]
*** fertilizer_ranges=[(52, 97), (54, 97), (15, 97), (52, 94), (54, 94), (15, 94)]
**** water_ranges=[(49, 56), (41, 41), (7, 97), (11, 97), (50, 56), (53, 97), (7, 97), (11, 97), (49, 56), (4, 41), (7, 97), (11, 97), (49, 56), (41, 41), (7, 94), (11, 94), (50, 56), (53, 94), (7, 94), (11, 94), (49, 56), (4, 41), (7, 94), (11, 94)]
***** light_ranges=[(25, 56), (42, 49), (25, 41), (34, 34), (88, 94), (18, 87), (88, 94), (18, 87), (25, 56), (43, 49), (25, 97), (46, 87), (88, 94), (18, 87), (88, 94), (18, 87), (25, 56), (42, 49), (88, 94), (18, 34), (88, 94), (18, 87), (88, 94), (18, 87), (25, 56), (42, 49), (25, 41), (34, 34), (88, 94), (18, 87), (88, 94), (18, 87), (25, 56), (43, 49), (25, 94), (46, 87), (88, 94), (18, 87), (88, 94), (18, 87), (25, 56), (42, 49), (88, 94), (18, 34), (88, 94), (18, 87), (88, 94), (18, 87)]
****** temperature_ranges=[(25, 76), (81, 92), (25, 63), (42, 76), (81, 85), (42, 63), (25, 76), (25, 44), (

[(0, 55),
 (0, 92),
 (60, 73),
 (26, 92),
 (74, 96),
 (70, 92),
 (73, 96),
 (69, 92),
 (60, 72),
 (25, 92),
 (60, 68),
 (26, 92),
 (0, 55),
 (0, 92),
 (60, 73),
 (43, 92),
 (74, 89),
 (70, 92),
 (73, 89),
 (69, 92),
 (60, 72),
 (42, 92),
 (60, 68),
 (43, 92),
 (0, 55),
 (0, 92),
 (60, 73),
 (26, 92),
 (60, 72),
 (25, 92),
 (26, 55),
 (26, 92),
 (60, 72),
 (25, 92),
 (60, 68),
 (26, 92),
 (0, 55),
 (0, 92),
 (60, 73),
 (35, 92),
 (60, 72),
 (34, 92),
 (35, 55),
 (35, 92),
 (60, 72),
 (34, 92),
 (60, 68),
 (35, 92),
 (60, 72),
 (56, 92),
 (61, 67),
 (57, 92),
 (0, 55),
 (0, 92),
 (69, 73),
 (65, 92),
 (74, 96),
 (56, 57),
 (73, 96),
 (56, 57),
 (60, 72),
 (45, 92),
 (60, 60),
 (46, 92),
 (74, 96),
 (56, 59),
 (73, 96),
 (56, 59),
 (0, 55),
 (0, 92),
 (73, 73),
 (69, 92),
 (60, 72),
 (56, 92),
 (61, 67),
 (57, 92),
 (0, 55),
 (0, 92),
 (69, 73),
 (65, 92),
 (74, 96),
 (56, 57),
 (73, 96),
 (56, 57),
 (60, 72),
 (45, 92),
 (60, 60),
 (46, 92),
 (74, 96),
 (56, 59),
 (73, 96),
 (56, 59),
 (

In [22]:
min(min(location) for location in test_location_ranges)

0